In [ ]:
import warnings

In [ ]:
import os
import math
import json

import dill

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, regularizers, ops

import zucaml as ml

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error
from sklearn.metrics import f1_score, precision_score, recall_score

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
class QuakeLoss(tf.keras.losses.Loss):
    def __init__(self, number_points, scale_values, name="quake_loss"):
        super().__init__(name=name)

        self.number_points = number_points
        self.mag_scaler = scale_values['mag'][1]
        self.coord_scaler = tf.constant([
            scale_values['x'][1],
            scale_values['y'][1],
            scale_values['z'][1],
        ], dtype=tf.float32)

    def call(self, y_true, y_pred):

        stacked_coord_scaler = tf.tile(
            tf.expand_dims(self.coord_scaler, axis=0),
            [tf.shape(y_true)[0], 1]
        )

        coords = {}
        mags = {}
        for flat_name, flat_y in {'y_true': y_true, 'y_pred': y_pred}.items():
            reshaped_y = tf.reshape(flat_y, (-1, self.number_points, 4))
            coords[flat_name] = reshaped_y[..., :3]
            mags[flat_name] = reshaped_y[..., 3]

        coords['y_true'], mags['y_true'] = tf.map_fn(
            lambda inputs: self.reorder_closest_points(inputs[0], inputs[1], inputs[2], inputs[3]),
            (coords['y_pred'], coords['y_true'], mags['y_true'], stacked_coord_scaler),
            fn_output_signature=(tf.float32, tf.float32)
        )

        loss = tf.square((coords['y_pred'] - coords['y_true']) / self.coord_scaler)

        loss = tf.reduce_mean(loss, axis=-1)

        loss += tf.square((mags['y_pred'] - mags['y_true']) / self.mag_scaler)

        loss = tf.reduce_mean(loss)

        return loss

    def reorder_closest_points(self, A, B, C, array_scaler):

        sorted_indices = []
        available_mask = tf.ones(tf.shape(B)[:1], dtype=tf.bool)
    
        A_scaled = A / array_scaler
        B_scaled = B / array_scaler
    
        for i in range(A.shape[0]):
    
            distances = tf.reduce_sum(tf.square(A_scaled[i] - B_scaled), axis=1)
    
            distances = tf.where(available_mask, distances, tf.constant(float('inf'), dtype=tf.float32))
    
            closest_index = tf.argmin(distances)
    
            sorted_indices.append(closest_index)
    
            available_mask = tf.tensor_scatter_nd_update(available_mask, [[closest_index]], [False])
    
        return tf.gather(B, tf.stack(sorted_indices)), tf.gather(C, tf.stack(sorted_indices))

class QuakeActivation(tf.keras.layers.Layer):
    def __init__(self, number_points, grid_values, **kwargs):
        super().__init__(**kwargs)

        self.number_points = number_points
        self.grid_values = grid_values
        self.sharpness = tf.Variable(-1, trainable=False, dtype=tf.float32)
        self.activation_coord = self.rounded_vals
        self.activation_mag = self.final_mag

    def call(self, inputs):

        reshaped = tf.reshape(inputs, (-1, self.number_points, 4))
        coords = reshaped[..., :3]
        mags = reshaped[..., 3]

        coord_acts = []
        for i, (coord_name, coord_vals) in enumerate(self.grid_values.items()): # python >= 3.7

            coord_acts.append(
                self.activation_coord(
                    coords[..., i],
                    coord_vals['min'],
                    coord_vals['res'],
                    self.sharpness,
                )
            )

        mags = self.activation_mag(mags)

        outputs = tf.stack([i for i in coord_acts] + [mags], axis=-1)
        outputs = tf.reshape(outputs, (-1, self.number_points * 4))

        return outputs

    def identity_coord(self, x, v_min, v_step, sharpness): # same signature
        return x

    def smooth_round(self, x, sharpness):
        return tf.sigmoid(sharpness * (x - tf.floor(x) - 0.5)) + tf.floor(x)

    def discrete_vals(self, x, v_min, v_step, sharpness):
        x = x - tf.constant(v_min, dtype=tf.float32)
        x = x / tf.constant(v_step, dtype=tf.float32)
        x = self.smooth_round(x, sharpness)
        x = x * tf.constant(v_step, dtype=tf.float32)
        x = x + tf.constant(v_min, dtype=tf.float32)
        return x

    def rounded_vals(self, x, v_min, v_step, sharpness): # same signature
        x = x - tf.constant(v_min, dtype=tf.float32)
        x = x / tf.constant(v_step, dtype=tf.float32)
        x = tf.round(x)
        x = x * tf.constant(v_step, dtype=tf.float32)
        x = x + tf.constant(v_min, dtype=tf.float32)
        x = tf.cast(tf.cast(x, dtype=tf.int32), dtype=tf.float32)
        return x

    def identity_mag(self, x): # same signature
        return x

    def relu_mag(self, x):
        return tf.where(x <= 0.0, 0.0, x)

    def final_mag(self, x):
        return tf.where(x <= 0.0, 0.0, x)

class DiscreteScheduler(keras.callbacks.Callback):
    def __init__(self, layer, scheduled_values):
        super().__init__()
        self.layer = layer
        self.sharp_values = scheduled_values

    def on_epoch_begin(self, epoch, logs=None):

        current_sharp = float(self.layer.sharpness)

        scheduled_sharp = self.get_from_schedule(epoch, current_sharp, self.sharp_values)

        ################################
        self.layer.activation_coord = self.layer.relu_mag
        ################################

        if scheduled_sharp < 0:
            self.layer.activation_coord = self.layer.identity_coord
            self.layer.sharpness.assign(scheduled_sharp)
        else:
            self.layer.activation_coord = self.layer.discrete_vals
            self.layer.sharpness.assign(scheduled_sharp)

    def get_from_schedule(self, epoch, default_or_previous_value, schedule):
        if epoch < schedule[0][0] or epoch > schedule[-1][0]:
            return default_or_previous_value
        for i in range(len(schedule)):
            if epoch == schedule[i][0]:
                return schedule[i][1]
        return default_or_previous_value

    def on_train_end(self, logs=None):
        self.layer.activation_coord = self.layer.rounded_vals
        self.layer.activation_mag = self.layer.final_mag

#### gold

In [ ]:
df_gold = pd.read_csv('data/gold.csv')

sort_order = ['date', 'x', 'y', 'z']

df_gold = df_gold.sort_values(sort_order, ascending=True).reset_index(drop=True)

ml.print_memory(df_gold)
df_gold[:5]

#### features

In [ ]:
discarded_features = [
    'energy',
]

target = 'target'
time_ref = 'date'
pid = 'zone_frame'
x_dim = 'x'
y_dim = 'y'
z_dim = 'z'

location_features = [x_dim, y_dim, z_dim]

unique_locations = {}
for dim in location_features:
    unique_locations[dim] = list(df_gold[dim].unique())

grid_info = {}
for i in location_features:
    grid_info[i] = {}
    all_res = set(np.diff(np.sort(df_gold[i].unique())))
    assert(len(all_res) == 1)
    grid_info[i]['res'] = int(min(all_res))
    grid_info[i]['min'] = int(df_gold[i].min())
    grid_info[i]['max'] = int(df_gold[i].max())

time_info = {}
time_res = set(np.diff(np.sort(df_gold[time_ref].unique())) / np.timedelta64(1, 'D'))
assert(len(time_res) == 1)
time_info['res'] = int(min(time_res))

discarded_features += [target, time_ref, pid] + location_features

remaining_features = [feature for feature in df_gold if feature not in discarded_features]
all_features = [feature for feature in remaining_features if 'energy|rolling.mean' in feature]
all_features += [feature for feature in remaining_features if 'energy|shift' in feature]

leftover_features = [feature for feature in remaining_features if feature not in all_features]
assert len(leftover_features) == 0, f'Leftover features'

print(f'Total features\t\t {len(all_features)}')

In [ ]:
n_quakes = df_gold[df_gold[target] > 0].groupby(time_ref).agg({pid: 'count'})[pid].max()

n_quakes

In [ ]:
ground_zero_calc = QuakeActivation(n_quakes, grid_info)

ground_zero = {
    x_dim: ground_zero_calc.rounded_vals(0, grid_info[x_dim]['min'], grid_info[x_dim]['res'], None).numpy(),
    y_dim: ground_zero_calc.rounded_vals(0, grid_info[y_dim]['min'], grid_info[y_dim]['res'], None).numpy(),
    z_dim: ground_zero_calc.rounded_vals(0, grid_info[z_dim]['min'], grid_info[z_dim]['res'], None).numpy(),
}

use_cross_val = False

scale_mag = True

add_timewise_section = False
timewise_number_layers = 4
timewise_number_filters = 2

spatialtime_number_layers = 32
spatialtime_number_filters = 16

fullyconnected_config = [2**10] * 2

batch_size = 8
epochs = 100

use_discrete_scheduler = True
SHARPNESS_SCHEDULE = [
    (0, -1e1),
]

early_stopping = callbacks.EarlyStopping(
    min_delta = 1e-3,
    patience = 10,
    restore_best_weights = True,
)

optimizer = keras.optimizers.Adam(
    learning_rate = 1e-5,
)

In [ ]:
number_unique_pid = df_gold[pid].nunique()
number_x_dim = df_gold[x_dim].nunique()
number_y_dim = df_gold[y_dim].nunique()
number_z_dim = df_gold[z_dim].nunique()
number_features = len(all_features)
number_timeframes = df_gold[time_ref].nunique()

# make sure the full dataset is 'squared and full'
assert(number_unique_pid == number_x_dim * number_y_dim * number_z_dim)
assert(len(df_gold) == number_timeframes * number_unique_pid)
assert(df_gold.groupby(pid).agg({time_ref: 'count'})[time_ref].nunique() == 1)
assert(df_gold.groupby([pid, time_ref]).agg({target: 'count'})[target].nunique() == 1)

print(f'Unique')
print(f'PID: \t\t{number_unique_pid:,d}')
print(f'X: \t\t{number_x_dim}')
print(f'Y: \t\t{number_y_dim}')
print(f'Z: \t\t{number_z_dim}')
print(f'Features: \t{number_features}')
print(f'Timeframe: \t{number_timeframes:,d}')

#### Split

In [ ]:
this_problem = ml.problems.MULTICLASS

df_train_val, df_test = ml.split_by_time_ref(df_gold, 0.9, target, time_ref, this_problem, True)

df_gold = []

if not use_cross_val:
    df_train, df_val = ml.split_by_time_ref(df_train_val, 1.0 - 0.1 / 0.9, target, time_ref, this_problem, True)
else:
    df_train = df_train_val

df_train_val = []

In [ ]:
def get_scaling_values(coord_and_mag):

    reshaped = np.reshape(coord_and_mag, (-1, n_quakes, 4))

    values_scaling = {}
    for i, coord in enumerate(location_features):
        coord_values = reshaped[..., i]
        values_scaling[coord] = (np.mean(coord_values), np.std(coord_values))

    if scale_mag:
        mag_values = reshaped[..., 3]
        values_scaling['mag'] = (np.mean(mag_values), np.std(mag_values))
    else:
        values_scaling['mag'] = 0.0, 1.0

    return values_scaling

def pad_group(group):

    padded = group.to_dict(orient='records')

    trailing = {
        x_dim: ground_zero[x_dim],
        y_dim: ground_zero[y_dim],
        z_dim: ground_zero[z_dim],
        target: 0
    }

    padded.extend([trailing] * (n_quakes - len(group)))

    return padded

def transform_df(df, features, n_x_dim, n_y_dim, n_z_dim, n_features, preprocess, xyz_scaler):

    transformed = df[features].copy()
    transformed[time_ref] = df[time_ref]
    transformed[pid] = df[pid]
    transformed[target] = df[target]
    transformed[x_dim] = df[x_dim]
    transformed[y_dim] = df[y_dim]
    transformed[z_dim] = df[z_dim]

    return pivot_df(transformed, features, n_x_dim, n_y_dim, n_z_dim, n_features, xyz_scaler)

def pivot_df(df, features, n_x_dim, n_y_dim, n_z_dim, n_features, xyz_scaler):

    # make sure order is correct and save unique_time_ref for later check
    df = df.sort_values(sort_order, ascending=True).reset_index(drop=True)
    unique_time_ref =  df[time_ref].unique()

    # make sure every pid has the same count of time_ref (rows, and then unique)
    assert(df.groupby(pid).agg({time_ref: 'count'})[time_ref].nunique() == 1)
    assert(df.groupby([pid, time_ref]).agg({target: 'count'})[target].nunique() == 1)

    n_unique_pid = df[pid].nunique()
    # make sure that unique zones IN THIS dataframe are the same as found in global
    # there is a previous assert that check global_unique_pid == n_x * n_y * n_z
    assert(n_unique_pid == n_x_dim * n_y_dim * n_z_dim)

    n_unique_time_ref = df[time_ref].nunique()
    # make sure that unique time frames IN THIS dataframe are correct
    assert(len(df) == n_unique_time_ref * n_unique_pid)

    # features
    np_features = np.asarray(df[features])
    np_features = np.reshape(np_features, (n_unique_time_ref, n_x_dim, n_y_dim, n_z_dim, n_features))

    # rows in features_pivot == rows in original / number unique pid [THIS CHECK IS DUPLICATED by check timeref same as global]
    assert(np_features.shape[0] == n_unique_time_ref == df.shape[0] / n_unique_pid)

    # check number of NaN is maitained
    assert(pd.DataFrame(np_features.reshape(-1, n_features)).isna().sum().sum() == df.isna().sum().sum())

    # target
    np_target = df.loc[:, [time_ref, x_dim, y_dim, z_dim, target]].copy()

    original_time_refs = list(np_target[time_ref].unique())

    np_target = np_target[np_target[target] > 0]

    result_time_refs = list(np_target[time_ref].unique())

    no_events = pd.DataFrame([
        {
            time_ref: i,
            x_dim: ground_zero[x_dim],
            y_dim: ground_zero[y_dim],
            z_dim: ground_zero[z_dim],
            target: 0,
        }
        for i in original_time_refs
        if i not in result_time_refs
    ])

    np_target = pd.concat([np_target, no_events])
    np_target = np_target.sort_values(sort_order, ascending=True)
    np_target = np_target.reset_index(drop=True)

    np_target = np_target.groupby(time_ref)[location_features + [target]].apply(pad_group)
    # check if they have the same (order and length) time_ref # this can fail if df_target is not padded time-wise
    assert(np.all(np_target.index == unique_time_ref))
    np_target = pd.DataFrame(np_target.to_list())
    np_target = pd.concat([pd.json_normalize(np_target[col]) for col in np_target], axis=1, ignore_index=True)
    np_target = np.asarray(np_target)

    # normalize location and mag
    if xyz_scaler is None:
        xyz_scaler = get_scaling_values(np_target)

    # check if resulting target shape is as expected: rows - one for each date, cols - possible quakes * (3D + mag)
    assert(np_target.shape == (n_unique_time_ref, n_quakes * 4))

    return np_features, np_target, xyz_scaler

In [ ]:
train_X, train_y, values_scaler = transform_df(
    df_train, all_features, number_x_dim, number_y_dim, number_z_dim, number_features, None, None
)

if not use_cross_val:
    val_X, val_y, _ = transform_df(
        df_val, all_features, number_x_dim, number_y_dim, number_z_dim, number_features, None, values_scaler
    )

test_X, test_y, _ = transform_df(
    df_test, all_features, number_x_dim, number_y_dim, number_z_dim, number_features, None, values_scaler
)

# make sure [first] all have the same shape and [second] the timeframe summed is equal to original
for dim in [1, 2, 3]:
    assert(train_X.shape[dim] == test_X.shape[dim])
    if not use_cross_val:
        assert(train_X.shape[dim] == val_X.shape[dim])
timeframe_summed = train_X.shape[0] + test_X.shape[0]
if not use_cross_val:
    timeframe_summed += val_X.shape[0]
assert(number_timeframes == timeframe_summed)

# make sure [first] all have the same shape and [second] the timeframe summed is equal to original
for dim in [1]:
    assert(train_y.shape[dim] == test_y.shape[dim])
    if not use_cross_val:
        assert(train_y.shape[dim] == val_y.shape[dim])
timeframe_summed = train_y.shape[0] + test_y.shape[0]
if not use_cross_val:
    timeframe_summed += val_y.shape[0]
assert(number_timeframes == timeframe_summed)

train_X.shape, values_scaler

#### NN

In [ ]:
def add_conv1d_block(number_filters, shape_kernel, l2_value, dropout_value, name_prefix):
    def _block(input_tensor):
        fx = layers.DepthwiseConv1D(
            kernel_size=shape_kernel,
            padding='valid',
            depth_multiplier=number_filters,
            depthwise_regularizer=regularizers.L2(l2_value),
            activation='relu',
            name=f'{name_prefix}_conv1d',
        )(input_tensor)
        fx = keras.layers.Dropout(dropout_value, name=f'{name_prefix}_dropout')(fx)
        fx = layers.BatchNormalization(name=f'{name_prefix}_batchnorm')(fx)
        return fx
    return _block

def add_conv3d_block(number_filters, shape_kernel, l2_value, dropout_value, name_prefix):
    def _block(input_tensor):
        fx = layers.Conv3D(
            filters=number_filters,
            kernel_size=shape_kernel,
            padding='same',
            kernel_regularizer=regularizers.L2(l2_value),
            activation='relu',
            name=f'{name_prefix}_conv3d',
        )(input_tensor)
        fx = keras.layers.Dropout(dropout_value, name=f'{name_prefix}_dropout')(fx)
        fx = layers.BatchNormalization(name=f'{name_prefix}_batchnorm')(fx)
        return fx
    return _block

In [ ]:
#####################################
# Input
#####################################
inputs = keras.Input(shape=(number_x_dim, number_y_dim, number_z_dim, number_features), name='input')
scale_layer = keras.layers.Normalization(axis=-1)
input_scaled = scale_layer(inputs)

#####################################
# Time-wise section
#####################################
first_section = input_scaled

if add_timewise_section:
    section_name = '1st_section_'

    first_section = layers.Reshape(
        (-1, number_features),
        name=f'{section_name}convert_to_1D',
    )(first_section)
    first_section = layers.Permute((2, 1), name=f'{section_name}zone_as_features')(first_section)

    for i in range(timewise_number_layers):
        new_block = add_conv1d_block(
            number_filters=timewise_number_filters if i == 0 else 1,
            shape_kernel=3,
            l2_value=1e-2,
            dropout_value=0.2,
            name_prefix=f'{section_name}{i + 1}',
        )
        first_section = new_block(first_section)

    first_section = layers.Permute((2, 1), name=f'{section_name}zone_as_spatial')(first_section)
    first_section = layers.Reshape(
        (number_x_dim, number_y_dim, number_z_dim, -1),
        name=f'{section_name}convert_back_to_3D',
    )(first_section)

#####################################
# Spatial-time section
#####################################
second_section = first_section

for i in range(spatialtime_number_layers):
    new_block = add_conv3d_block(
        number_filters=spatialtime_number_filters,
        shape_kernel=3,
        l2_value=1e-2,
        dropout_value=0.2,
        name_prefix=f'2nd_section_{i + 1}',
    )
    second_section = new_block(second_section)

#####################################
# FC section
#####################################
fc_section = layers.Flatten(name='fc_section_flat')(second_section)

for i, number_neurons in enumerate(fullyconnected_config):
    fc_section = layers.Dense(number_neurons, activation='relu', name=f'fc_section_{i + 1}')(fc_section)

#####################################
# Output section
#####################################
outputs = layers.Dense(n_quakes * 4, activation=None)(fc_section)
final_activation_layer = QuakeActivation(n_quakes, grid_info)
outputs = final_activation_layer(outputs)

#####################################
# Create
#####################################
model = keras.Model(inputs=inputs, outputs=outputs, name='quake_net')

#####################################
# Compile
#####################################
model.compile(
    optimizer = optimizer,
    loss = QuakeLoss(n_quakes, values_scaler),
)

model.summary()

train

In [ ]:
%%time

callbacks_to_use = [early_stopping]
if use_discrete_scheduler:
    callbacks_to_use.append(DiscreteScheduler(final_activation_layer, SHARPNESS_SCHEDULE))

fit_params = {
    'x': train_X,
    'y': train_y,
    'batch_size': batch_size,
    'epochs': epochs,
    'callbacks': callbacks_to_use,
}

if use_cross_val:
    fit_params['validation_split'] = 0.1
else:
    fit_params['validation_data'] = (val_X, val_y)

history = model.fit(**fit_params)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

history_df[history_df['val_loss'] == history_df['val_loss'].min()]

#### Score model

In [ ]:
sets = {
    'train': (df_train, train_X, train_y),
    'test': (df_test, test_X, test_y),
}

flats = {}

for set_name, (df, np_X, np_y) in sets.items():
    
    score_df = (
        pd
        .DataFrame(df[time_ref].unique(), columns=[time_ref])
        .sort_values(time_ref, ascending=True)
        .reset_index(drop=True)
    )

    y_pred = model.predict(np_X)
    y_true = np_y.copy()

    flats[set_name] = (y_pred, y_true, score_df)

In [ ]:
def expand_pred(preds):

    def expand_rows(row):
        expanded_rows = []
        for col in location_features:
            for add_sub in [1, -1]:
                new_row_add = row.copy()
                new_row_add[col] += add_sub * grid_info[col]['res']
                expanded_rows.append(new_row_add)
        return expanded_rows
    
    all_rows = []

    # TODO: improve this, although this is not long
    for _, row in preds.iterrows():
        all_rows.append(row)
        all_rows.extend(expand_rows(row))

    expanded = pd.DataFrame(all_rows, columns=preds.columns).reset_index(drop=True)
    expanded[location_features] = expanded[location_features].astype(int)

    return expanded

# order is important
grand_metrics = {
    1: 'tp',
    2: 'total_true_positives',
    3: 'total_pred_positives',
}
def create_grand_metrics(df):
    for grand_metric_k in sorted(grand_metrics):
        df[grand_metrics[grand_metric_k]] = 0
    return df

def insert_grand_metrics(df, i, metrics_values):
    for metric_k, metric in grand_metrics.items():
        df.iloc[i, metric_k] = metrics_values[metric]
    return df

def reshape_flats(preds, trues):
    coordinates = {}
    magnitudes = {}
    for flat_name, flat_y in {'y_true': trues, 'y_pred': preds}.items():

        reshape_y = np.reshape(flat_y, (flat_y.shape[0], n_quakes, 4))
        
        coordinates[flat_name] = reshape_y[..., :3]
        magnitudes[flat_name] = reshape_y[..., 3]

    return coordinates, magnitudes

def calculate_tps(coords_pred, mags_pred, coords_true, mags_true, margin_tp, margin_fp, expand_preds):

    res_metrics = {}

    res_true = pd.DataFrame(coords_true)
    res_true = pd.concat([res_true, pd.DataFrame(mags_true)], axis=1)
    res_true.columns = location_features + ['mag_true']
    res_true = res_true[res_true['mag_true'] > 0].reset_index(drop=True)
    res_true[location_features] = res_true[location_features].astype(int)
    res_metrics['total_true_positives'] = len(res_true)

    res_pred = pd.DataFrame(coords_pred)
    res_pred = pd.concat([res_pred, pd.DataFrame(mags_pred)], axis=1)
    res_pred.columns = location_features + ['mag_pred']
    res_pred = res_pred[res_pred['mag_pred'] > 0].reset_index(drop=True)
    res_pred[location_features] = res_pred[location_features].astype(int)
    res_metrics['total_pred_positives'] = len(res_pred[res_pred['mag_pred'] > margin_fp])
    if expand_preds:
        res_pred = expand_pred(res_pred)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res_true = res_true.merge(
            res_pred,
            on=location_features,
            how='left',
        )

    res_true['mag_abs_diff'] = (res_true['mag_pred'] - res_true['mag_true']).abs()

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res_metrics['tp'] = (res_true['mag_abs_diff'] < margin_tp).sum()

    return res_metrics

def calculate_scores(margin_tp, margin_fp, expand_predictions = False):

    epsilon = np.finfo(float).eps

    scores = {}
    for set_name, (y_pred, y_true, score_df) in flats.items():

        scores[set_name] = {}
        coords, mags = reshape_flats(y_pred, y_true)
        score_df = create_grand_metrics(score_df)

        for i in range(y_pred.shape[0]):

            metrics_values = calculate_tps(
                coords_pred=coords['y_pred'][i],
                mags_pred=mags['y_pred'][i],
                coords_true=coords['y_true'][i],
                mags_true=mags['y_true'][i],
                margin_tp=margin_tp,
                margin_fp=margin_fp,
                expand_preds=expand_predictions,
            )

            score_df = insert_grand_metrics(score_df, i, metrics_values)

        for i in grand_metrics.values():
            scores[set_name][i] = score_df[i].sum()

    for set_name in flats:
        scores[set_name]['fn'] = scores[set_name]['total_true_positives'] - scores[set_name]['tp']
        scores[set_name]['fp'] = scores[set_name]['total_pred_positives'] - scores[set_name]['tp']
        precision = scores[set_name]['tp'] / (scores[set_name]['tp'] + scores[set_name]['fp'] + epsilon)
        recall = scores[set_name]['tp'] / (scores[set_name]['tp'] + scores[set_name]['fn'] + epsilon)
        scores[set_name]['precision'] = precision
        scores[set_name]['recall'] = recall

        scores[set_name]['f1_score'] = 2 * precision * recall / (precision + recall + epsilon)

    scores['test']['overfit'] = scores['train']['f1_score'] - scores['test']['f1_score']

    return scores

calculate_scores(1, 1)['test']